In [ ]:
!pip install typing-extensions --upgrade
# Import Libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
from torch.utils.data import DataLoader
from torch.cuda.amp import autocast, GradScaler
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import time


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

if device.type == 'cuda':
    torch.backends.cudnn.benchmark = True


transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
])

batch_size = 128


trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)


class ResNetCIFAR10(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNetCIFAR10, self).__init__()
        self.resnet = resnet18(weights="IMAGENET1K_V1")
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, num_classes)

    def forward(self, x):
        return self.resnet(x)


model = ResNetCIFAR10().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01, steps_per_epoch=len(trainloader), epochs=20)

scaler = GradScaler() 


def train(model, trainloader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct, total = 0, 0
    loop = tqdm(trainloader, desc="Training", leave=False)

    for inputs, labels in loop:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

      
        with autocast():
            outputs = model(inputs)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        loop.set_postfix(loss=loss.item(), acc=100.0 * correct / total)

    return running_loss / len(trainloader), 100.0 * correct / total


def evaluate(model, testloader, criterion, device):
    model.eval()
    correct, total = 0, 0
    running_loss = 0.0

    with torch.inference_mode():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    accuracy = 100.0 * correct / total
    return running_loss / len(testloader), accuracy



epochs = 20
best_acc = 0.0

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    
    train_loss, train_acc = train(model, trainloader, optimizer, criterion, device)
    test_loss, test_acc = evaluate(model, testloader, criterion, device)
    scheduler.step()

    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
    print(f"Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.2f}%\n")

    # Save the best model
    if test_acc > best_acc:
        best_acc = test_acc
        torch.save(model.state_dict(), "best_model.pth")
        print("Model Saved!")

print(f"Best Test Accuracy: {best_acc:.2f}%")



model.load_state_dict(torch.load("best_model.pth", map_location=device))
model.eval()



def imshow(img):
    img = img / 2 + 0.5  # Unnormalize
    npimg = img.cpu().numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


dataiter = iter(testloader)
images, labels = next(dataiter)

imshow(torchvision.utils.make_grid(images[:8]))


images, labels = images.to(device), labels.to(device)
outputs = model(images)
_, predicted = torch.max(outputs, 1)


print('Predicted: ', ' '.join(f'{predicted[j].item()}' for j in range(8)))
print('Actual:    ', ' '.join(f'{labels[j].item()}' for j in range(8)))


You should consider upgrading via the 'C:\Users\uppal\anaconda3\python.exe -m pip install --upgrade pip' command.


Using device: cpu


100%|███████████████████████████████████████████████████████████████████████████████| 170M/170M [00:22<00:00, 7.56MB/s]
C:\Users\uppal\AppData\Local\Temp/ipykernel_14492/1820668432.py:63: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()  # For mixed precision training
C:\Users\uppal\anaconda3\lib\site-packages\torch\amp\grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


Epoch 1/20


Training:   0%|                                                                                | 0/391 [00:00<?, ?it/s]C:\Users\uppal\AppData\Local\Temp/ipykernel_14492/1820668432.py:78: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
C:\Users\uppal\anaconda3\lib\site-packages\torch\amp\autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


...

Train Loss: 0.3625 | Train Acc: 87.39%
Test Loss: 0.4514 | Test Acc: 85.29%

Epoch 17/20


Train Loss: 0.3556 | Train Acc: 87.69%
Test Loss: 0.4564 | Test Acc: 85.13%

Epoch 18/20


Train Loss: 0.3506 | Train Acc: 87.91%
Test Loss: 0.4715 | Test Acc: 84.62%

Epoch 19/20


Train Loss: 0.3388 | Train Acc: 88.18%
Test Loss: 0.4466 | Test Acc: 85.13%

Epoch 20/20


Train Loss: 0.3300 | Train Acc: 88.54%
Test Loss: 0.4531 | Test Acc: 84.98%

Best Test Accuracy: 85.43%
